In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np 
import os 

/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

image_dir = "../sample_image/pracimages/org"

# Dataset 디렉토리에 있는 파일들의 리스트를 저장 
imgname_list = [os.path.join(image_dir, name) for name in os.listdir(image_dir)]
target_value = [weight[-7:-4] for weight in os.listdir(image_dir)]

imgname_queue = tf.train.string_input_producer(imgname_list, num_epochs=1, shuffle=False,)

img_reader = tf.WholeFileReader()

# key_img:파일이름, values_img:데이터  return 
key_img, raw_img = img_reader.read(imgname_queue)

decoded_img = tf.image.decode_png(raw_img)

# gray image reshaping 
mean_reduced_image = tf.reduce_mean(decoded_img, axis=-1)
reshaped_img = tf.reshape(mean_reduced_image, [-1])

weight = tf.placeholder(tf.float32)

In [3]:
with tf.Session() as sess :
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess, coord)

    sess.run(tf.global_variables_initializer())
    
    img_dir = '../sample_image/preprocessed/train.tfrecord'
    img_test_dir = '../sample_image/preprocessed/test.tfrecord'
    
    train_writer = tf.python_io.TFRecordWriter(img_dir)
    test_writer = tf.python_io.TFRecordWriter(img_test_dir)
    
    for i in range(31):
        try :
            _img, _key, _weight = sess.run([reshaped_img, key_img, weight, {weight : target_value}])
            
            example = tf.train.Example()
            example.features.feature['weight'].int64_list.value.append(_weight[0])
            example.features.feature['image'].int64_list.value.extend(_img.tolist())
            
            if i < 21:
                train_writer.write(example.SerializeToString())
                print('{} train data has been written'.format(i))
            else:
                test_writer.write(example.SerializeToString())
                print('{} test data has been written'.format(i))
        
        except tf.errors.OutOfRangeError:
            print('size of total dataset {}'.format(i))
            break
    
    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(thread)

ValueError: Fetch argument '113' cannot be interpreted as a Tensor. ("The name '113' refers to an Operation not in the graph.")